In [43]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='2a3c97b4-9a50-4a49-acbf-460288594b85', project_access_token='p-2+30RA4NO9tZEN6d5sgJn6LA==;M4ELCcLBQ29rCEf/jzU6fw==:ag7Uq7E48Qr4HjUgBPXlbEqgTJ/uaWGwFaA5/IRZf+vzgJqge1dj+Xov7ZDv+B2Y4ogdwH77Nw1FBgTq/om2NQ0vGs/Co7ZeRw==')
pc = project.project_context

from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space({'token':'p-2+30RA4NO9tZEN6d5sgJn6LA==;M4ELCcLBQ29rCEf/jzU6fw==:ag7Uq7E48Qr4HjUgBPXlbEqgTJ/uaWGwFaA5/IRZf+vzgJqge1dj+Xov7ZDv+B2Y4ogdwH77Nw1FBgTq/om2NQ0vGs/Co7ZeRw=='})

In [44]:
# Install and Import the NLTK library
!pip install nltk
import nltk
# Download the Punkt tokenizer from the NLTK repository
nltk.download('punkt')
# Download the stopwords corpus from the NLTK repository
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/wsuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/wsuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [45]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='rX5ySY4M7bEzYU0eSIrFIxwJOaknsea8I2m0oFm8uWeP',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'sacarcarrentingapplication-donotdelete-pr-flygcd2aonfzrf'
object_key = 'train.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_train = pd.read_csv(body)


object_key = 'test.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_test = pd.read_csv(body)

In [46]:
print(df_train.shape, df_test.shape)

(100, 11) (50, 11)


# Analyzing Car Complaints with Watson NLP

This notebook demonstrates how to analyze car complaints using Watson NLP.

The data that is used in this notebook is taken from the National Highway and Transit Association (NHTSA) of the US Department of Transportation. The NHTSA records complaints from car owners, and makes them available regularly.  

## What you'll learn in this notebook

Watson NLP offers _blocks_ for various NLP tasks. This notebooks shows:

- **Syntax analysis** with _Syntax block_ for English (`syntax_izumo_en_stock`). This block extracts nouns and their _lemma_ (base form) from the car complaints. This way, for the nouns _wheels_ and _wheel_, the block collects _wheel_. The most frequently used noun lemmas are typical problems that review authors talk about.
- **Noun phrase extraction** with _Noun Phrases block_ for English (`noun-phrases_rbr_en_stock`). This block extracts larger noun phrases from car complaints to yield additional insights. 
- **How to correlate structured data with the analysis results**, with _association rule mining_ . This is used to find the most significant nouns for a particular car model. 


## Table of Contents

1. [Load the car complaints](#loadData)
1. [Analyze complaint descriptions with Watson NLP](#analyzeComplaints)
1. [Explore and analyze the results](#exploreResults)
1. [Identify nouns and noun phrases that are _specific_ to a given Model with Association Rule Mining](#correlate_n_nph)
1. [Summary](#summary)

Begin by importing the Watson NLP library that is used throughout the notebook.

In [47]:
import watson_nlp

<a id="loadData"></a>
## Load the car complaints

The data set used in this notebook from the National Highway and Transit Association (NHTSA) contains complaint records from car owners. The data sample provides information about the make and model of the car, the trouble-causing component, and a brief description of the issue. It also records the company's response to the issue described by the customer, as well as the customer's overall satisfaction level. The data set contains a total of 32 columns and 558 rows.

Load the car complaints into a dataframe.

In [48]:
# Split the data into training and test sets, keeping only the Customer_Service and Satisfaction columns
df_train = df_train[['Customer_Service', 'Satisfaction']]
df_test = df_test[['Customer_Service', 'Satisfaction']]

### Embed the text columns

Check if there is any watson embedding technique available

In [49]:
# Split the data into training and test sets.
train_X, train_y = df_train['Customer_Service'], df_train['Satisfaction']
test_X, test_y = df_test['Customer_Service'], df_test['Satisfaction']

In [50]:
# Get the shape of the 'Satisfaction' column in the training dataframe.
df_train['Satisfaction'].shape

(100,)

In [51]:
train_X.shape, train_y.shape

((100,), (100,))

In [52]:
# Embed the data
import gensim
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd

In [53]:

def preprocess(text: str) -> str:
    # Remove punctuations, numbers and stop words
    text = re.sub(r'[^\w\s]', '', text)
    number_pattern = re.compile(r'\d+')
    text = re.sub(number_pattern, '', text)
    stp_wrds = stopwords.words('english')
    text = ' '.join([word for word in text.split() if word not in stp_wrds])
    #Converts to lowercase
    return text.lower()

In [54]:
# Preprocess the text in the train_X and test_X DataFrames
train_X = train_X.apply(preprocess)
test_X = test_X.apply(preprocess)

In [180]:
from gensim.models import Word2Vec
embedding_size = 128

sentences = pd.concat([train_X, test_X])

In [181]:
model = Word2Vec(sentences, vector_size=embedding_size, window=2, min_count=2)

In [182]:
def get_word2vec_embeddings(text, model):
    text_embedding = np.zeros(embedding_size)
    vectors = []
    word_count = 0
    for word in text:
        # TODO: Maybe take padding or use GloVe
        text_embedding = text_embedding + model.wv[word]
        word_count += 1
    text_embedding = text_embedding / word_count
    return text_embedding

In [183]:
train_X = train_X.apply(lambda item: get_word2vec_embeddings(item, model))
test_X = test_X.apply(lambda item: get_word2vec_embeddings(item, model))

In [184]:
train_X = np.concatenate(train_X).reshape(100, 128)
test_X = np.concatenate(test_X).reshape(50, 128)

In [200]:
from sklearn.metrics import f1_score, precision_score, recall_score

def get_metrics(pred, true) -> dict:
    return {
        'f1-score': f1_score(pred, true),
        'precision': precision_score(pred, true),
        'recall': recall_score(pred, true)
    }

In [201]:
# Build Random Forest and Naive Bayes Model
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
clf.fit(train_X, train_y)

pred_y = clf.predict(test_X)

get_metrics(pred_y, test_y)

{'f1-score': 0.742857142857143,
 'precision': 0.8125,
 'recall': 0.6842105263157895}

In [202]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_X, train_y)

pred_y = gnb.predict(test_X)
get_metrics(pred_y, test_y)

{'f1-score': 0.5384615384615384, 'precision': 0.4375, 'recall': 0.7}

pandas.core.series.Series

## Authors

__Alexander Lang__ IBM, Germany

__Simone Zerfass__ IBM, Germany

# <hr>
Copyright © 2021 IBM. This notebook and its source code are released under the terms of the MIT License.